In [11]:
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.layers import SimpleRNN, LSTM
from tensorflow.keras.optimizers import RMSprop, Adam
import tensorflow as tf
import datetime
import pandas as pd
import seaborn as sns
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import rcParams
import matplotlib as mpl
import time
import csv
from collections import OrderedDict
%matplotlib inline

sns.set()
rcParams['figure.figsize'] = 20, 10
rcParams['path.simplify_threshold'] = 1.0

### Tensorflow 버전 확인

In [12]:
print(tf.__version__)

from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

2.1.0
[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 12069479427065463358
]


### 데이터 로드 및 전처리

In [27]:
dataset = pd.read_csv('example.csv', sep=',')

In [28]:
dataset['datetime'] = pd.to_datetime(dataset['RECORD_TIME'], format = "%Y-%m-%d %H:%M:%S+00:00")
dataset = dataset[['BATTERY_VOLTAGE', 'datetime']]
dataset.head()

,BATTERY_VOLTAGE,datetime
0,286,2020-06-15 00:24:04
1,286,2020-06-15 00:24:05
2,286,2020-06-15 00:24:06
3,282,2020-06-15 00:24:07
4,282,2020-06-15 00:24:08


In [29]:
resultdict = dataset.to_dict('series')
resultdict.keys()

dict_keys(['BATTERY_VOLTAGE', 'datetime'])

# [MAIN] 시간차 데이터셋 생성
    두 데이터 사이의 시간차를 계산함

In [31]:
sdate = {0:0}
for time in range(len(resultdict['datetime'])-1):
    sdate[time] = (resultdict['datetime'][time+1] - resultdict['datetime'][time+1]).total_seconds()

resultdict['subdate'] = pd.Series(sdate)

### 데이터셋 생성을 위한 변수

In [34]:
x_dataset = []
y_dataset = []
dateset = []

### LSTM 모델 변수 설정

    lookback: 입력으로 사용하기 위해 거슬러 올라갈 타임스텝(학습 시 데이터 갯수)
    delay: 타깃으로 사용할 미래의 타임스텝
    batch_size: 배치 샘플 수 
    step: 데이터를 샘플링할 타임스템 간격

In [35]:
lookback = 60
delay = 30
batch_size = 100
step = 1

# [MAIN] 데이터셋 생성 코드
    끊긴부분을 넘어가며 데이터셋을 생성함. 데이터가 끊겼을 시 다음 데이터셋 부분으로 넘어감.

In [36]:
loc = 0
check=1
while loc < len(resultdict['BATTERY_VOLTAGE']) - (lookback+delay+1):
    #데이터 연속성 검사 - 뒤에서부터 iter를 돌리며 끊긴 지점이 있을 시 해당 죄표로 loc를 이동
    #cut = 새로 시작될 위치, check = 완성 여부
    check=1
    for cut in range(lookback+delay):
        new_loc = loc + (lookback+delay) - cut
        if(resultdict['subdate'][new_loc]>1):
            loc = new_loc
            check=0
            break
    if check==1:
        x_dataset.append(resultdict['BATTERY_VOLTAGE'][loc:loc+30].tolist())
        y_dataset.append(resultdict['BATTERY_VOLTAGE'][loc+60])
        dateset.append(loc)
        loc+=1

In [38]:
print(len(y_dataset), '/', len(dataset), '데이터 생성')

26618 / 26709 데이터 생성


### 데이터 형태 변환

In [39]:
x_dataset = np.array(x_dataset)
y_dataset = np.array(y_dataset)

In [40]:
print('x_shape:', x_dataset.shape)
print('y_shape:', y_dataset.shape)

x_shape: (26618, 30)
y_shape: (26618,)


### 데이터 분할

In [46]:
n = 10000

train_x_dataset = x_dataset[0:n,:]
train_y_dataset = y_dataset[0:n,]

test_x_dataset = x_dataset[n:-1,:]
test_y_dataset = y_dataset[n:-1,]

In [47]:
train_x_dataset = train_x_dataset.reshape((train_x_dataset.shape[0], train_x_dataset.shape[1], 1))
test_x_dataset = test_x_dataset.reshape((test_x_dataset.shape[0], test_x_dataset.shape[1], 1))

### 네트워크 구성
    사용자의 모델에 맞게 네트워크 구성

In [48]:
model_LSTM = Sequential()
model_LSTM.add(LSTM(50,
                    input_shape=(30,1),
                    return_sequences=True,
                    dropout=0.1,
                    recurrent_dropout=0.5))
model_LSTM.add(LSTM(50,
                    dropout=0.1,
                    return_sequences=True,
                    recurrent_dropout=0.5))
model_LSTM.add(LSTM(50,
                    dropout=0.1,
                    return_sequences=True,
                    recurrent_dropout=0.5))
model_LSTM.add(LSTM(50,
                    dropout=0.1,
                    recurrent_dropout=0.5))
model_LSTM.add(layers.Dense(1))

model_LSTM.compile(optimizer=RMSprop(), loss='mse')

In [49]:
history_LSTM = model_LSTM.fit(train_x_dataset, train_y_dataset, epochs=2, batch_size=30)

Train on 10000 samples
Epoch 1/2
10000/10000 [==============================] - 39s 4ms/sample - loss: 70543.2814
Epoch 2/2
10000/10000 [==============================] - 30s 3ms/sample - loss: 61647.7459
